## Configuration
_Initial steps to get the notebook ready to play nice with our repository. Do not delete this section._

Code formatting with [black](https://pypi.org/project/nb-black/).

In [12]:
%load_ext lab_black

The lab_black extension is already loaded. To reload it, use:
  %reload_ext lab_black


In [13]:
import os
import pytz
import glob
import pathlib

this_dir = pathlib.Path(os.path.abspath(""))
data_dir = this_dir / "data"

In [14]:
import requests
import pandas as pd
import regex as re
import urllib, json
from datetime import datetime
from bs4 import BeautifulSoup

## Download

Retrieve the page

In [15]:
POWER_BI_QUERY_URL = (
    "https://wabi-us-gov-iowa-api.analysis.usgovcloudapi.net/public/reports/querydata"
)
REQUEST_PARAMS = (("synchronous", "true"),)
REQUEST_BODY = '{"version":"1.0.0","queries":[{"Query":{"Commands":[{"SemanticQueryDataShapeCommand":{"Query":{"Version":2,"From":[{"Name":"s1","Entity":"sde COVID19BYZIPCODES","Type":0}],"Select":[{"Column":{"Expression":{"SourceRef":{"Source":"s1"}},"Property":"zipcode"},"Name":"sde COVID19BYZIPCODES.zipcode"},{"Aggregation":{"Expression":{"Column":{"Expression":{"SourceRef":{"Source":"s1"}},"Property":"covid_cases"}},"Function":0},"Name":"Sum(sde COVID19BYZIPCODES.covid_cases)"},{"Column":{"Expression":{"SourceRef":{"Source":"s1"}},"Property":"Rank_Total (bins)"},"Name":"sde COVID19BYZIPCODES.Rank_Total (bins)"}],"OrderBy":[{"Direction":2,"Expression":{"Aggregation":{"Expression":{"Column":{"Expression":{"SourceRef":{"Source":"s1"}},"Property":"covid_cases"}},"Function":0}}}]},"Binding":{"Primary":{"Groupings":[{"Projections":[0,1]}]},"Secondary":{"Groupings":[{"Projections":[2]}]},"DataReduction":{"DataVolume":4,"Primary":{"Top":{}},"Secondary":{"Top":{}}},"Version":1},"ExecutionMetricsKind":1}}]},"CacheKey":"{\\"Commands\\":[{\\"SemanticQueryDataShapeCommand\\":{\\"Query\\":{\\"Version\\":2,\\"From\\":[{\\"Name\\":\\"s1\\",\\"Entity\\":\\"sde COVID19BYZIPCODES\\",\\"Type\\":0}],\\"Select\\":[{\\"Column\\":{\\"Expression\\":{\\"SourceRef\\":{\\"Source\\":\\"s1\\"}},\\"Property\\":\\"zipcode\\"},\\"Name\\":\\"sde COVID19BYZIPCODES.zipcode\\"},{\\"Aggregation\\":{\\"Expression\\":{\\"Column\\":{\\"Expression\\":{\\"SourceRef\\":{\\"Source\\":\\"s1\\"}},\\"Property\\":\\"covid_cases\\"}},\\"Function\\":0},\\"Name\\":\\"Sum(sde COVID19BYZIPCODES.covid_cases)\\"},{\\"Column\\":{\\"Expression\\":{\\"SourceRef\\":{\\"Source\\":\\"s1\\"}},\\"Property\\":\\"Rank_Total (bins)\\"},\\"Name\\":\\"sde COVID19BYZIPCODES.Rank_Total (bins)\\"}],\\"OrderBy\\":[{\\"Direction\\":2,\\"Expression\\":{\\"Aggregation\\":{\\"Expression\\":{\\"Column\\":{\\"Expression\\":{\\"SourceRef\\":{\\"Source\\":\\"s1\\"}},\\"Property\\":\\"covid_cases\\"}},\\"Function\\":0}}}]},\\"Binding\\":{\\"Primary\\":{\\"Groupings\\":[{\\"Projections\\":[0,1]}]},\\"Secondary\\":{\\"Groupings\\":[{\\"Projections\\":[2]}]},\\"DataReduction\\":{\\"DataVolume\\":4,\\"Primary\\":{\\"Top\\":{}},\\"Secondary\\":{\\"Top\\":{}}},\\"Version\\":1},\\"ExecutionMetricsKind\\":1}}]}","QueryId":"","ApplicationContext":{"DatasetId":"a58dcee6-4ec9-4c44-8344-b6e60bd89b34","Sources":[{"ReportId":"c9ddc766-a793-4781-9194-5669ce5c3b0a","VisualId":"50ec15fff2b157300ff9"}]}}],"cancelQueries":[],"modelId":333663}'

In [16]:
headers = {
    "Host": "wabi-us-gov-iowa-api.analysis.usgovcloudapi.net",
    "Connection": "keep-alive",
    "DNT": "1",
    "RequestId": "17ef22cc-113e-94c8-ba66-d2691e841d14",
    "ActivityId": "4d457d90-f83b-481b-a7e5-5a4cc7a1c85a",
    "Accept": "application/json, text/plain, */*",
    "X-PowerBI-ResourceKey": "f2e447ca-7ccb-4a1a-a234-683368ca5ce8",
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.81 Safari/537.36",
    "Content-Type": "application/json;charset=UTF-8",
    "sec-ch-ua-platform": "macOS",
    "Origin": "https://app.powerbigov.us",
    "Sec-Fetch-Site": "cross-site",
    "Sec-Fetch-Mode": "cors",
    "Sec-Fetch-Dest": "empty",
    "Referer": "https://app.powerbigov.us/",
    "Accept-Encoding": "gzip, deflate, br",
    "Accept-Language": "en-US,en;q=0.9,da;q=0.8",
}
json_data = json.loads(REQUEST_BODY)

In [17]:
response = requests.post(
    POWER_BI_QUERY_URL,
    params=REQUEST_PARAMS,
    headers=headers,
    json=json_data,
    verify=False,
)

/Users/pdo/.local/share/virtualenvs/california-coronavirus-scrapers-l_auzzjX/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'wabi-us-gov-iowa-api.analysis.usgovcloudapi.net'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [18]:
data = response.json()

In [29]:
data["results"][0]

{'jobId': 'be956bba-a5f8-4a77-bedc-4be0e15fc757',
 'result': {'data': {'descriptor': {'Select': [{'Kind': 1,
      'Depth': 0,
      'Value': 'G0',
      'GroupKeys': [{'Source': {'Entity': 'sde COVID19BYZIPCODES',
         'Property': 'zipcode'},
        'Calc': 'G0',
        'IsSameAsSelect': True}],
      'Name': 'sde COVID19BYZIPCODES.zipcode'},
     {'Kind': 2,
      'Value': 'M0',
      'Name': 'Sum(sde COVID19BYZIPCODES.covid_cases)'},
     {'Kind': 1,
      'SecondaryDepth': 0,
      'Value': 'G1',
      'Format': '0',
      'GroupKeys': [{'Source': {'Entity': 'sde COVID19BYZIPCODES',
         'Property': 'Rank_Total (bins)'},
        'Calc': 'G1',
        'IsSameAsSelect': True}],
      'Name': 'sde COVID19BYZIPCODES.Rank_Total (bins)'}],
    'Expressions': {'Primary': {'Groupings': [{'Keys': [{'Source': {'Entity': 'sde COVID19BYZIPCODES',
           'Property': 'zipcode'},
          'Select': 0}],
        'Member': 'DM0'}]},
     'Secondary': {'Groupings': [{'Keys': [{'Source

## Parse

In [20]:
response.headers

{'Cache-Control': 'no-store, must-revalidate, no-cache', 'Pragma': 'no-cache', 'Content-Length': '744', 'Content-Type': 'text/plain; charset=utf-8', 'Content-Encoding': 'gzip', 'X-PowerBI-Cache-Age': '3600', 'Strict-Transport-Security': 'max-age=31536000; includeSubDomains', 'X-Frame-Options': 'deny', 'X-Content-Type-Options': 'nosniff', 'RequestId': '17ef22cc-113e-94c8-ba66-d2691e841d14', 'Access-Control-Allow-Origin': '*', 'Access-Control-Expose-Headers': 'requestId,retry-after', 'Date': 'Thu, 21 Oct 2021 22:00:41 GMT'}

In [21]:
cities = data["features"]

KeyError: 'features'

In [ ]:
dict_list = []

In [ ]:
for item in cities:
    d = dict(item["attributes"])
    dict_list.append(d)

In [ ]:
df = pd.DataFrame(dict_list)

Rename fields to standardized column headers

In [ ]:
df = df.rename(columns={"city_district": "area", "confirmed": "confirmed_cases"})

Eliminate unneeded rows

In [ ]:
df = df.filter(["area", "confirmed_cases"], axis=1).sort_values(
    by="area", ascending=True
)

Drop where confirmed cases is NaN

In [ ]:
df = df.dropna(subset=["confirmed_cases"])

Convert `confirmed_cases` column to int from float

In [ ]:
df = df.astype({"confirmed_cases": int})

Sum districts 1-5 into unincorporated / districts

In [ ]:
total = df[
    df["area"].isin(
        ["District 1", "District 2", "District 3", "District 4", "District 5"]
    )
]

Append districts sum row to the dataframe

In [ ]:
new_row = {
    "area": "Unincorporated districts",
    "confirmed_cases": total["confirmed_cases"].sum(),
}
df = df.append(new_row, ignore_index=True)

Delete District rows

In [ ]:
df.drop(
    df[
        df["area"].isin(
            ["District 1", "District 2", "District 3", "District 4", "District 5"]
        )
    ].index,
    inplace=True,
)

In [ ]:
df = df.reset_index(drop=True)

### Adding county column

In [ ]:
df.insert(0, "county", "Stanislaus")

Get county timestamp (using separate feed to extract unix timestamp)

In [ ]:
url_date = "https://gis2.stancounty.com/arcgis/rest/services/COVID19_Cases_City_Zip_Layer/FeatureServer/0/metadata"
r_date = requests.get(url_date)
data_date_bad = BeautifulSoup(r_date.text).find("creadate").text
data_date = datetime.strptime(data_date_bad, "%Y%m%d").strftime("%Y-%m-%d")
# dropping last three digits of unix timestamp, converting to python date
feed_date_obj = data_date

If separate feed or modified date unavailable, use now for date

In [ ]:
field_date = data_date if data_date else now

In [ ]:
df["county_date"] = pd.to_datetime(field_date).date()

## Vet

In [ ]:
default_stan_len = 10

In [ ]:
try:
    assert not len(df) < default_stan_len
except AssertionError:
    raise AssertionError(
        "Stanislaus County scraper: latest spreadsheet entry is missing row(s)"
    )

In [ ]:
try:
    assert not len(df) > default_stan_len
except AssertionError:
    raise AssertionError(
        "Stanislaus County scraper: latest spreadsheet entry has more area(s) than previously reported"
    )

## Export

Set date

In [ ]:
tz = pytz.timezone("America/Los_Angeles")

In [ ]:
today = datetime.now(tz).date()

In [ ]:
slug = "stanislaus"

In [ ]:
df.to_csv(data_dir / slug / f"{today}.csv", index=False)

## Combine

In [ ]:
csv_list = [
    i
    for i in glob.glob(str(data_dir / slug / "*.csv"))
    if not str(i).endswith("timeseries.csv")
]

In [ ]:
df_list = []
for csv in csv_list:
    if "manual" in csv:
        df = pd.read_csv(csv, parse_dates=["date"])
    else:
        file_date = csv.split("/")[-1].replace(".csv", "")
        df = pd.read_csv(csv, parse_dates=["county_date"])
        df["date"] = file_date
    df_list.append(df)

In [ ]:
df = pd.concat(df_list).sort_values(["date", "area"])

In [ ]:
df.to_csv(data_dir / slug / "timeseries.csv", index=False)